In [1]:
from __future__ import annotations

import os
import pickle

import numpy as np
from qtpyt.base.selfenergy import DataSelfEnergy as BaseDataSelfEnergy
from qtpyt.block_tridiag import greenfunction
from qtpyt.continued_fraction import get_ao_charge
from qtpyt.projector import ProjectedGreenFunction, expand
from scipy.optimize import root

from edpyt.dmft import DMFT, Converged, Gfimp
from edpyt.nano_dmft import Gfimp as nanoGfimp
from edpyt.nano_dmft import Gfloc

In [2]:
def distance(delta):
    global delta_prev
    delta_prev[:] = delta
    return dmft.distance(delta)


def save_sigma(sigma_diag, outputfile, npsin):
    L, ne = sigma_diag.shape
    sigma = np.zeros((ne, L, L), complex)

    def save(spin):
        for diag, mat in zip(sigma_diag.T, sigma):
            mat.flat[:: (L + 1)] = diag
        np.save(outputfile, sigma)

    for spin in range(nspin):
        save(spin)

class DataSelfEnergy(BaseDataSelfEnergy):
    """Wrapper"""

    def retarded(self, energy):
        return expand(S_molecule, super().retarded(energy), idx_molecule)


def load(filename):
    return DataSelfEnergy(energies, np.load(filename))

In [ ]:
U = 4.0  # Interaction
nbaths = 4
tol = 1.5
max_iter = 4
alpha = 0.0
nspin = 1
mu = U/2
eta = 3e-2
data_folder = "../output/compute_run"
output_folder = "../output/compute_run/model"
os.makedirs(output_folder, exist_ok=True)

occupancy_goal = np.load(f"{data_folder}/occupancies.npy")
energies = np.arange(-10,10,0.01)
z_ret = energies + 1.j * eta

H_active = np.load(f"{data_folder}/hamiltonian.npy").real
index_active_region = np.load(f"{data_folder}/index_active_region.npy")
len_active = len(index_active_region)

beta = 1000
ne = 3000
z_mats = 1.0j * (2 * np.arange(ne) + 1) * np.pi / beta

HybMats = lambda z: 0.0

S_active = np.eye(len_active)

idx_neq = np.arange(len_active)
idx_inv = np.arange(len_active)

V = np.eye(len_active) * U


In [6]:
gfloc = Gfloc(H_active, np.eye(len_active), HybMats, idx_neq, idx_inv)
gfloc.mu = mu
nimp = gfloc.idx_neq.size
gfimp = []
for i in range(nimp):
    gfimp.append(Gfimp(nbaths, z_mats.size, V[i, i], beta))

gfimp = nanoGfimp(gfimp)

occupancy_goal = occupancy_goal[gfloc.idx_neq]

dmft = DMFT(
    gfimp,
    gfloc,
    occupancy_goal,
    max_iter=max_iter,
    tol=tol,
    adjust_mu=False,
    alpha=alpha,
    store_iterations=True,
    egrid = z_ret,
    iter_filename = f"{output_folder}/dmft_iterations.h5"
)

Sigma = lambda z: np.zeros((nimp, z.size), complex)
delta = dmft.initialize(V.diagonal().mean(), Sigma, mu=mu)
delta_prev = delta.copy()


In [7]:
try:
    root(distance, delta_prev, method="broyden1")
except Converged:
    pass

np.save(f"{output_folder}/dmft_delta.npy", delta_prev)
open(f"{output_folder}/mu.txt", "w").write(str(gfloc.mu))

_Sigma = lambda z: -gfloc.mu + gfloc.Sigma(z)[idx_inv]


Iteration :  0
Occupation : 11.70173 | Chemical potential : 2.00000 | Relative Error : 1.70839
Iteration :  1
Occupation : 11.70173 | Chemical potential : 2.00000 | Relative Error : 2.05770
Iteration :  2
Occupation : 11.70173 | Chemical potential : 2.00000 | Relative Error : 1.68716
Iteration :  3
Occupation : 11.70173 | Chemical potential : 2.00000 | Relative Error : 0.97553


In [8]:
dmft_sigma_file = f"{output_folder}/dmft_sigma.npy"
save_sigma(_Sigma(z_ret), dmft_sigma_file, nspin)

In [9]:
self_energy = np.load(f"{data_folder}/self_energy.npy",allow_pickle=True)
with open(f"{data_folder}/hs_list_ii.pkl", 'rb') as f:
    hs_list_ii = pickle.load(f)
with open(f"{data_folder}/hs_list_ij.pkl", 'rb') as f:
    hs_list_ij = pickle.load(f)


In [10]:
gf = greenfunction.GreenFunction(hs_list_ii,
                                hs_list_ij,
                                [(0, self_energy[0]),
                                (len(hs_list_ii) - 1, self_energy[1])],
                                solver='dyson',
                                eta=eta)

In [ ]:
gfp = ProjectedGreenFunction(gf, index_active_region)
charge = get_ao_charge(gfp)


TypeError: expected str, bytes or os.PathLike object, not ndarray

In [13]:
np.save(f"{output_folder}/charge_per_orbital.npy",charge)

In [ ]:

nodes = [0,810,1116,1278,1584,2394]
imb = 2  # index of molecule block from the nodes list
S_molecule = hs_list_ii[imb][1]  # overlap of molecule
idx_molecule = index_active_region - nodes[imb]  # indices of active region w.r.t molecule

dmft_sigma = load(dmft_sigma_file)
self_energy[2] = dmft_sigma
gf.selfenergies.append((imb, self_energy[2]))

In [ ]:
gfp_dmft = ProjectedGreenFunction(gf, index_active_region)
charge_dmft = get_ao_charge(gfp_dmft)
np.save(f"{output_folder}/charge_per_orbital_dmft.npy",charge_dmft)


In [6]:
import h5py

with h5py.File(f'{output_folder}/dmft_iterations.h5', 'r') as f:
    # Access last n iterations of delta, sigma, gfloc
    last_n_data = f['last_n_iterations']
    for iter_key in last_n_data.keys():
        delta = last_n_data[f'{iter_key}/delta'][:]
        sigma = last_n_data[f'{iter_key}/sigma'][:]
        gfloc = last_n_data[f'{iter_key}/gfloc'][:]


with h5py.File(f'{output_folder}/dmft_iterations.h5', 'r') as f:
    # Access all bath parameters for each iteration
    bath_data = f['all_bath_parameters']
    for iter_num in sorted(bath_data.keys()):
        iter_grp = bath_data[f'{iter_num}']
        print(f"Iteration: {iter_num}")
        for bath_idx in range(9):
            vk = iter_grp[f'vk_{bath_idx}'][:]
            ek = iter_grp[f'ek_{bath_idx}'][:]
            print(f"  Bath {bath_idx}: vk = {vk}, ek = {ek}")


Iteration: iteration_0
  Bath 0: vk = [ 1.43522381 -1.36049928  0.89997291], ek = [-5.08651834  1.00525302  4.37930194]
  Bath 1: vk = [2.22809143 2.24950774 2.39296756], ek = [-4.17125751 -0.22343322  4.24314158]
  Bath 2: vk = [2.49163781 3.27539316 2.0429297 ], ek = [-5.86868235 -1.63353989  4.24070351]
  Bath 3: vk = [1.33862676 4.00614949 2.26647876], ek = [-5.26663992 -1.47762222  4.03964537]
  Bath 4: vk = [1.69282536 1.85705493 1.35507076], ek = [-4.44278437  0.88447045  5.02733713]
  Bath 5: vk = [2.28315188 2.51573504 1.91649074], ek = [-3.91622504  0.28929369  4.51794002]
  Bath 6: vk = [2.56018674 3.2826961  2.05363419], ek = [-5.73566162 -1.62745845  4.16028754]
  Bath 7: vk = [2.22701504 2.24063312 2.41937638], ek = [-4.19204059 -0.2057461   4.24328419]
  Bath 8: vk = [1.48866322 1.39725609 0.96083441], ek = [-5.04878137  1.00842968  4.22216073]
Iteration: iteration_1
  Bath 0: vk = [ 1.42873674 -1.19190648  1.08437698], ek = [-4.34130491  1.03505639  4.05236112]
  Bath 1